In [38]:
import sys
sys.path.append('../')

from src.features.graphs import pipe_get_graph_dependency
from transformers import BertTokenizer, BertModel
import torch

In [39]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [41]:
text = "my name is mansur"
nodes = pipe_get_graph_dependency([text])[0][0]
print(nodes)

['my', 'name', 'is', 'mansur']


In [42]:
node_ids = bert_tokenizer.convert_tokens_to_ids(nodes)

node_ids

[2026, 2171, 2003, 100]

In [43]:
input_tensor = torch.tensor([node_ids])

# make embedding

with torch.no_grad():
    last_hidden_states = bert_model(input_tensor)[0]  # Models outputs are now tuples
    print(last_hidden_states.shape)
    print(last_hidden_states)

torch.Size([1, 4, 768])
tensor([[[-0.5262,  0.0037, -0.2047,  ..., -0.0646,  0.4079,  0.4193],
         [-0.2162,  0.3499,  0.1357,  ..., -0.0970,  0.4418,  0.0676],
         [-0.4372,  0.3469,  0.1108,  ..., -0.0828,  0.4179,  0.2761],
         [-0.6043,  0.4287,  0.1793,  ..., -0.1501,  0.4506,  0.1635]]])


In [19]:
# average tokens which have ## in them

embeddings = []
i = 0

while i < len(nodes):
    embedding = [last_hidden_states[0][i]]
    
    i += 1
    while i < len(nodes) and "##" in nodes[i]:
        embedding.append(last_hidden_states[0][i])
        i += 1

    embeddings.append(torch.mean(torch.stack(embedding), dim=0))

embeddings = torch.stack(embeddings)

print(embeddings)

tensor([[-0.0542,  0.1198,  0.0206,  ..., -0.2848,  0.4606,  0.4578],
        [-0.3199,  0.1903, -0.2605,  ..., -0.6791,  0.7809,  0.2531],
        [-0.2987,  0.2777,  0.0491,  ..., -0.4080,  0.6818,  0.8072],
        ...,
        [-0.6011,  0.1999,  0.1491,  ..., -0.1202,  0.2963,  0.4673],
        [-0.5147,  0.0944, -0.2155,  ...,  0.0167,  0.5935,  0.6680],
        [-0.7929,  0.4672, -0.4117,  ..., -0.3198,  0.4967,  0.2549]])
